### _Nothing in Biology Makes Sense Except in the Light of Evolution_ 
###### Theodosius  Dobzhansky

# TP-1: alineamiento global con Needleman Wunsch

Escriba una implementación en Python del algoritmo de alineamiento global de Needleman-Wunsch, 
dentro de las 4 celdas indicadas más abajo.

El alineamiento entre las 2 secuencias de `ab.fasta` debe tener un score de 158 y
el alineamiento entre las 2 secuencias de `ae.fasta` debe tener un score de 312.

--------

In [12]:
import numpy as np
from Bio import pairwise2, SeqIO
from enum import Enum

## Needleman-Wunsch con BioPython

In [13]:
aln = pairwise2.align.globalxx("TATA", "TCT")
aln[0].score

2.0

In [14]:
print(aln[0].seqA)
print(aln[0].seqB)

TA-TA
T-CT-


In [15]:
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)

seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)

aln = pairwise2.align.globalxx(seq_A, seq_B)
aln[0].score

158.0

In [16]:
print(aln[0].seqA[300:])
print(aln[0].seqB[300:])

CGTCA-AAGATCTC-CAA-CCCGGGGACAGAGT-ACTGGCCGCGGATT-ACGACGGAA--ACCCGGTTTATACCGACTTCATCATGTTCAA-
C-TC-CAA-AT-T-ACAATCCC---GAC--A-TTA-T-----C---TTTA--A-GG-ATGA---GG-----A--GA----A-CA----C--G


In [17]:
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

aln = pairwise2.align.globalxx(seq_A, seq_E)
aln[0].score

312.0

In [18]:
print(aln[0].seqA[300:])
print(aln[0].seqB[300:])

GGCCCG-TCAAAGATC-TCCAACCCGGG-GACAGAG-TAC-TGGCC-GCG-GATT-ACGA-CGGAAACCCGG-TTTAT-ACCGACTTCATCATGTTCAA
GG--CGGTCAAAGA-CCTCCAACCC-GGCGACAG-GGT-CCTGG-CAGC-AGA-TGA-GAACGGAAACCC-GATTTA-CACCGAC-TC--C---TT---


------

## Needleman-Wunsch propio

#### Cosas q pueden ser útiles

### 1
`op` es un type que representa las 3 operaciones posibles en un alineamiento.

In [19]:
class op(Enum):
    GAP_A = 1
    GAP_B = 2
    MA_MM = 3

### 2
`score_mtx` contiene los scores de match y mismatch entre las distintas bases.
Para reproducir el comportamiento de la función`pairwise2.align.globalxx()` de biopython
asignaremos, por ahora, un score de `1` a cualquier match y `0` a cualquier otro match.

El gap score irá por fuera de `score_mtx` y será, circunstancialmente, de `0`.

`score_mtx` puede ser una lista de listas, `numpy.darray`, diccionario, o lo que crea conveniente.

In [57]:
score_mtx = np.repeat(-1, (4) * (4)).reshape(4, 4)
for i in range(0,4):
    score_mtx[i][i]=1
score_mtx

array([[ 1, -1, -1, -1],
       [-1,  1, -1, -1],
       [-1, -1,  1, -1],
       [-1, -1, -1,  1]])

### 3
`traceback()` toma la tabla donde `nw()` memoizó los subproblemas (`tabla_sol`),
las 2 secuencias originales (`seqA` y `seqB`) y entrega las 2 secuencias modificadas (`aln_a` y `aln_b`),
que puestas una arriba de la otra, mostrarían un alineamiento correcto.

In [73]:
def traceback(matrix_score, seqA, seqB):
    steps, operations = shorter_path(matrix_score, 0,0)
    operations.reverse()
    print(operations)
    return align(seqA, seqB,operations,0, 0, 0)

def shorter_path(matrix_score, x, y):
    lenX, lenY = matrix_score.shape
    if(x == lenX and y == lenY):
        return 0,[]
    if(x == lenX-1):
        return lenY-y, [ 'gap_A' for x in range(lenY-1-y)]
    if(y == lenY-1):
        return lenX-x,[ 'gap_B' for x in range(lenX-1-x)]
    if matrix_score[x][y] == matrix_score[x+1][y+1]:
        rightSteps, rightOperations   = shorter_path(matrix_score,x+1,y)
        leftSteps, leftOperations = shorter_path(matrix_score,x,y+1)
        if rightSteps < leftSteps:
            return rightSteps+1, rightOperations + ['gap_A']
        return leftSteps+1, leftOperations + ['gap_B']
    diagonalSteps, diagonalOperations = shorter_path(matrix_score,x+1,y+1)
    return diagonalSteps + 1, diagonalOperations + ['Match']

def align(seqA,seqB,operations,actual_op_id,x,y):
    lenA = len(seqA) 
    lenB = len(seqB) 
    if (x == lenA and y == lenB):
        return '', ''
    if x == lenA:
        return '', seqB[y:]
    if y == lenB:
        return  seqA[x:], ''
    actual_op = operations[actual_op_id]
    if actual_op == 'Match':
        aln_a_res, aln_b_res= align(seqA,seqB,operations,actual_op_id+1,x+1,y+1)
        return seqA[x] + aln_a_res, seqB[y] + aln_b_res
    if actual_op == 'gap_A':
        aln_a_res, aln_b_res= align(seqA,seqB,operations,actual_op_id+1,x,y+1)
        return '-' + aln_a_res, seqB[y] + aln_b_res
    if actual_op == 'gap_B':
        aln_a_res, aln_b_res= align(seqA,seqB,operations,actual_op_id+1,x+1,y)
        return seqA[x] + aln_a_res, '-' + aln_b_res

In [74]:
def parseNucleotide(nucleotide):
    switcher = {
        'A': 0,
        'T': 1,
        'G': 2,
        'C': 3
    }
    return switcher.get(nucleotide,"Invalid nucleotide")

In [75]:
def get_score(score_mtx,characterA,characterB):
        indexA = parseNucleotide(characterA)
        indexB = parseNucleotide(characterB)
        return score_mtx[indexA,indexB]

### 4
`nw()` implementa el algoritmo de needleman-wunsch. Toma las 2 secuencias en formato de `str` (`seqA_str` y `seqB_str`),
para igualar el comportamiento de `pairwise2.align.globalxx()`, la `score_mtx` y un `gap_score`.
Debe llamar a `traceback()` para luego devolver las 2 secuencias alineadas y el score del alineamiento (último elemento de `tabla_score`)

In [76]:
def nw(seqA_str, seqB_str, score_mtx, gap_score = 0):
    sizeA = len(seqA_str)
    sizeB = len(seqB_str)
    sol_matrix = np.repeat(0, (sizeA+1) * (sizeB+1)).reshape(sizeA+1, sizeB+1)
    
    for i in range(1,sizeA+1):
        for j in range(1,sizeB+1):
            
            diagonal_score = get_score(score_mtx,seqA_str[i-1],seqB_str[j-1])
            diagonal_result = sol_matrix[i-1][j-1] + diagonal_score
            if diagonal_score == 1:
                sol_matrix[i][j] = diagonal_result
            else:
                left_result = sol_matrix[i][j-1] + gap_score
                above_result = sol_matrix[i-1][j] + gap_score
                sol_matrix[i][j] = max(left_result,above_result + gap_score , diagonal_result)  
    print(sol_matrix)
    aln_a, aln_b = traceback(sol_matrix,seqA_str,seqB_str)

    return aln_a, aln_b, sol_matrix[sizeA, sizeB]

nw("TATA","TCA",score_mtx,0)

In [77]:
nw("TATA","TCA",score_mtx,0)

[[0 0 0 0]
 [0 1 1 1]
 [0 1 1 2]
 [0 1 1 2]
 [0 1 1 2]]


IndexError: index 5 is out of bounds for axis 0 with size 5

## Pruebe su implementación

In [ ]:
# Toma de input
f_aln = SeqIO.parse("ab.fasta", 'fasta')
fas_A = next(f_aln)
fas_B = next(f_aln)

seq_A = str(fas_A.seq)
seq_B = str(fas_B.seq)

In [ ]:
# Confirme el score
aln_a, aln_b, score = nw(seq_A, seq_B, score_mtx_1)
score

In [ ]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_b = str().join(aln_b)

print(str_aln_a[300:])
print(str_aln_b[300:])

In [ ]:
# Toma de input
f_aln = SeqIO.parse("ae.fasta", 'fasta')
fas_A = next(f_aln)
fas_E = next(f_aln)

seq_A = str(fas_A.seq)
seq_E = str(fas_E.seq)

In [ ]:
# Confirme el score
aln_a, aln_e, score = nw(seq_A, seq_E, score_mtx_1)
score

In [ ]:
# Visualice el alineamiento. Recuerde que no es necesario que sea idéntico al de Biopython
str_aln_a = str().join(aln_a)
str_aln_e = str().join(aln_e)

print(str_aln_a[300:])
print(str_aln_e[300:])